In [15]:
import os
from pygame import mixer
from gtts import gTTS

my_mp3 = "my.mp3"
my_txt_file = "speach.txt"

# my_file = open("speach.txt","r")
# my_text = my_file.read()
# my_file.close()
with open(my_txt_file,"w") as my_file:
   my_file.write("Hello world again") 
with open(my_txt_file,"r") as my_file:
   my_text = my_file.read() 

mixer.init()
tts = gTTS(text = my_text,lang = "en")
tts.save(my_mp3)
mixer.music.load(my_mp3)
mixer.music.play()
os.remove(my_mp3)
# import pyaudio
# import speech_recognition as sr


win+r
cmd
pip install py

In [9]:
pip --version

pip 23.0 from /Users/pattycha/Library/Python/3.8/lib/python/site-packages/pip (python 3.8)
Note: you may need to restart the kernel to use updated packages.
